In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [3]:
# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.common.by import By

In [4]:
# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

Fase 2: Extracción de Detalles de Películas con Selenium
Una vez que tengan una lista de películas de la API, el siguiente paso es obtener información más detallada sobre ellas. Utilizaras la herramienta de automatización web Selenium para navegar a sitios web de reseñas de películas (IMDB y Rotten Tomatoes) y extraer detalles como calificaciones, actores y directores. En concreto deberás extraer información sobre:

Puntuacion de IMDB (en caso de que la tenga).

Puntuación de Rotten Tomatoes (Tomatometer).

Dirección (director/a o directore/as de cada película).

Gionistas (de cada película).

Argumento.

Duración (en minutos).

Nombre de la película

In [7]:
# Llamo al navegador

driver=webdriver.Chrome()
driver.get("https://www.imdb.com/?ref_=nv_home")
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

# Hago grande el navegador para que aparezcan todas las opciones posibles de la web

driver.maximize_window()

sleep(5)


# Creamos diccionario para guardar la informacion

diccionario_info_pelis={"puntuacion":[],
                        "direccion":[], 
                        "guionistas":[], 
                        "argumento":[], 
                        "duracion":[], 
                        "nombre_pelicula":[] }


# Accedemos a la primera pelicula de la lista de la API que es "Mom´s coming"

lista_id_muestra= ['tt10579986','tt10597228','tt11097384','tt11152168','tt11237950']

for id in lista_id_muestra:

    try:

      busqueda = driver.find_element("css selector","#suggestion-search")

      busqueda.clear()

      busqueda.send_keys(id, Keys.ENTER)



      #driver.find_element("css selector","#suggestion-search").send_keys(Keys.ENTER)


      # Añadimos un poco de scroll para que se vea mejor

      #driver.execute_script("window.scrollTo(0,250)")

      # Extraemos la informacion que necesitamos

      nombre_pelicula=driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-b7c53eda-0.dUpRPQ > h1 > span").text 
      #nombre_pelicula = driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-4e4cc5f9-3.dDRspk > div.sc-b7c53eda-0.dUpRPQ > h1 > span").text
      diccionario_info_pelis["nombre_pelicula"].append(nombre_pelicula)

      direccion_peli= driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > ul > li:nth-child(1) > div > ul > li > a").text
      diccionario_info_pelis["direccion"].append(direccion_peli)

      guionistas_peli= driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > ul > li:nth-child(2) > div > ul > li > a").text
      diccionario_info_pelis["guionistas"].append(guionistas_peli)

      
      argumento_peli=driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p/span[1]').get_attribute("class")
      #argumento_peli= driver.find_element("xpath", '/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[3]/div[2]/div[1]/div/div/div').text
      diccionario_info_pelis["argumento"].append(argumento_peli) #__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(28) > div.sc-f65f65be-0.bBlII > ul > li:nth-child(1) > div

      etiqueta2= "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-b7c53eda-0.dUpRPQ > ul > li:nth-child(2)"
      
      try: 
         duracion_peli=driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-b7c53eda-0.dUpRPQ > ul > li:nth-child(3)").text
         #duracion_peli= driver.find_element("css selector","__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(28) > div.sc-f65f65be-0.bBlII > ul > li:nth-child(1) > div").text
         diccionario_info_pelis["duracion"].append(duracion_peli)

      except:
         duracion_peli=driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-b7c53eda-0.dUpRPQ > ul > li:nth-child(2)").text
         diccionario_info_pelis["duracion"].append(duracion_peli)

      #puntuacion = driver.find_element
      #diccionario_info_pelis[""].append() 

    except Exception as null:
        print(f"Error al procesar la búsqueda para el ID {id}: {str(null)}")

    driver.back()

Error al procesar la búsqueda para el ID tt11237950: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-b7c53eda-0.dUpRPQ > ul > li:nth-child(2)"}
  (Session info: chrome=124.0.6367.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7C6BB1502+60802]
	(No symbol) [0x00007FF7C6B2AC02]
	(No symbol) [0x00007FF7C69E7CE4]
	(No symbol) [0x00007FF7C6A36D4D]
	(No symbol) [0x00007FF7C6A36E1C]
	(No symbol) [0x00007FF7C6A7CE37]
	(No symbol) [0x00007FF7C6A5ABBF]
	(No symbol) [0x00007FF7C6A7A224]
	(No symbol) [0x00007FF7C6A5A923]
	(No symbol) [0x00007FF7C6A28FEC]
	(No symbol) [0x00007FF7C6A29C21]
	GetHandleVerifier [0x00007FF7C6EB411D+3217821]


In [8]:
diccionario_info_pelis

{'puntuacion': [],
 'direccion': ['Rishiking',
  'Erik Krefeld',
  'Johan Renck',
  'John Krasinski',
  'Frederick Keeve'],
 'guionistas': ['Rishiking',
  'Stevie Kinchen',
  'Jaroslav Kalfar',
  'John Krasinski',
  'Frederick Keeve'],
 'argumento': ['sc-7193fc79-0 ftEVcu',
  'sc-7193fc79-0 ftEVcu',
  'sc-7193fc79-0 ftEVcu',
  'sc-7193fc79-0 ftEVcu',
  'sc-7193fc79-0 ftEVcu'],
 'duracion': ['2h 2min', '1h 30min', '1h 47min', '1h 44min'],
 'nombre_pelicula': ["Mom's Coming",
  'Love Song and Power',
  'El astronauta',
  'Amigos imaginarios',
  'The Accompanist Awakening']}